## URL extraction

In [27]:
import pandas as pd
import requests
import numpy as np 
from bs4 import BeautifulSoup
import re

data = pd.read_csv('sample_port.csv')

names = data['Scheme Name']



def check_url_for_404(url):
    try:
        response = requests.get(url)
        if response.status_code == 404:
            return True
        else:
            return False
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return False

# Example usage

scheme_url = []
NAV_list = []

for name in names:
    name1 = name.lower().replace(' ','-').replace('---','-').replace('regular','direct')
    
    
    name2 = name1.replace('plan','growth')

    # remove the string after growth 
    name3 = name2.split('growth', 1)[0].replace('-direct-','-direct-growth').replace('--','-').replace('-idcw-reinvestment','')

    if 'fund-of-fund' in name3:
        name3 = name3.replace('fund-of-fund','fof')

    if name3.endswith('fund-'):
        name3 = name3.replace('fund-','fund-direct-growth')
    
    if name3.endswith('fund'):
        name3 = name3.replace('fund','fund-direct-growth')
    
    if name3.endswith('discovery-'):
        name3 = name3.replace('discovery-','discovery-fund-direct-growth')
    
    if name3=='icici-prudential-large-&-mid-cap-fund-direct-growth':
        name3 = 'icici-prudential-top-100-fund-direct-growth'
    
    if name3=='icici-prudential-bluechip-fund-direct-growth':
        name3 = 'icici-prudential-focused-bluechip-equity-fund-direct-growth' 
    
    if name3=='sbi-large-&-midcap-fund-direct-growth':
        name3 = 'sbi-magnum-multiplier-fund-direct-growth'

    if name3=='kotak-emerging-equity-fund':
        name3 = 'kotak-emerging-equity-scheme-direct-growth'

    if name3=='quant-small-cap-fund-direct-growth':
        name3 = 'quant-small-cap-fund-direct-plan-growth'

    if name3=='dsp-flexi-cap-fund-direct-growth':
        name3 = 'dsp-blackrock-equity-fund-direct-growth'

    if name3=='kotak-emerging-equity-fund-direct-growth':
        name3 = 'kotak-emerging-equity-scheme-direct-growth'

    if name3=='kotak-small-cap-fund-direct-growth':
        name3= 'kotak-midcap-fund-direct-growth'

    if name3=='templeton-india-equity-income-fund-direct-growth':
        name3 = 'templeton-india-equity-i-come-fund-direct-growth'

    if name3=='dsp-banking-&-financial-services-fund-direct-growth':
        name3 = 'dsp-banking-financial-services-fund-direct-growth'

    url = f"https://groww.in/mutual-funds/{name3}"
    print(url)
    is_404 = check_url_for_404(url)
    if is_404:
        print(name3)

        print(f"The URL {url} returns a 404 status code.")
    
    urldat = requests.get(url)

    soup = BeautifulSoup(urldat.text, 'html.parser')



    td_tag = soup.find_all('td',class_="fd12Cell contentPrimary bodyLargeHeavy")[0]

    # Extract the text content from the <td> tag
    value = td_tag.get_text(strip=True)

    NAV= re.search(r'₹([\d.]+)', value).group(1)

    NAV_list.append(NAV)
    scheme_url.append(url)

    #.split('-')[0].lower().replace(' ','-')[:-1])

data['scheme_url'] = np.array(scheme_url)
data['NAV'] = np.array(NAV_list)

data.to_csv('sample_port.csv',index=False)


https://groww.in/mutual-funds/quant-momentum-fund-direct-growth
https://groww.in/mutual-funds/hdfc-transportation-and-logistics-fund-direct-growth
https://groww.in/mutual-funds/quant-momentum-fund-direct-growth
https://groww.in/mutual-funds/axis-india-manufacturing-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-top-100-fund-direct-growth
https://groww.in/mutual-funds/quant-commodities-fund-direct-growth
https://groww.in/mutual-funds/kotak-healthcare-fund-direct-growth
https://groww.in/mutual-funds/hdfc-mid-cap-opportunities-fund-direct-growth
https://groww.in/mutual-funds/hdfc-technology-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-focused-bluechip-equity-fund-direct-growth
https://groww.in/mutual-funds/franklin-india-flexi-cap-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-multicap-fund-direct-growth
https://groww.in/mutual-funds/nippon-india-multi-cap-fund-direct-growth
https://groww.in/mutual-funds/groww-nifty-total-market-i

## Data 

In [1]:
import pandas as pd 

data = pd.read_csv('sample_port.csv')

columns_to_keep = ['Scheme Name', 'ISIN', 'Scheme Category Name', 'Units', 'NAV','scheme_url']

data = data[columns_to_keep]

data.to_csv('sample_port.csv',index=False)
data.head()

,Scheme Name,ISIN,Scheme Category Name,Units,NAV,scheme_url
0,quant Momentum Fund - Regular Plan IDCW - Rein...,INF966L01CY6,Equity - Diversified,3000.000,13.73,https://groww.in/mutual-funds/quant-momentum-f...
1,HDFC Transportation and Logistics Fund Regular...,INF179KC1GX4,Equity - Sector - Auto,1500.000,13.49,https://groww.in/mutual-funds/hdfc-transportat...
2,quant Momentum Fund - Regular Plan Growth - Gr...,INF966L01CW0,Equity - Diversified,7075.965,13.73,https://groww.in/mutual-funds/quant-momentum-f...
3,Axis India Manufacturing Fund Regular Growth,INF846K013S0,Equity - Diversified,3000.000,11.75,https://groww.in/mutual-funds/axis-india-manuf...
4,ICICI Prudential Large & Mid Cap Fund - Growth,INF109K01431,Equity - Diversified,220.345,924.99,https://groww.in/mutual-funds/icici-prudential...


In [13]:
url = requests.get(data.loc[data['Scheme Name'] == scheme,'scheme_url'].values[0])
soup = BeautifulSoup(url.text, 'html.parser')

# string between > and &
soup.find_all('title')[0].get_text().split('-')[0].strip()
# remove trailing space


'HDFC Mid Cap Opportunities Direct Plan Growth'

In [14]:
print(soup.prettify())

<!DOCTYPE html>
<html data-theme="light" lang="en" style="color-scheme:light">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, minimum-scale=1" name="viewport"/>
  <link href="/favicon32x32-groww.ico" rel="icon"/>
  <title>
   HDFC Mid Cap Opportunities Direct Plan Growth - NAV, Mutual Fund Performance &amp; Portfolio
  </title>
  <meta content="HDFC Mid Cap Opportunities Direct Plan Growth - Get latest NAV, SIP Returns &amp; Rankings, Ratings, Fund Performance, Portfolio, Expense Ratio, Holding Analysis, and Peers.
    Invest in HDFC Mid-Cap Opportunities Fund Online with Groww." name="description"/>
  <meta content="index" name="robots"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="https://groww.in/groww-logo-270.png" name="twitter:image"/>
  <meta content="HDFC Mid Cap Opportunities Direct Plan Growth - Get latest NAV, SIP Returns &amp; Rankings, Ratings, Fund Performance, Portfolio, Expense Ratio, Holding Anal

In [19]:
import json
# Find the script tag with the specific ID
script_tag = soup.find('script', id='__NEXT_DATA__')

# Extract the JSON data from the script tag content
json_data = json.loads(script_tag.contents[0])

# get holdings
json_data['props']['pageProps']['mf']['sub-category']



KeyError: 'sub-category'

## Data get the holding

In [2]:
import requests
from bs4 import BeautifulSoup
import json


data = pd.read_csv('sample_port.csv')
all_schemes = data['Scheme Name'].unique()

consol_df = pd.DataFrame()
for scheme in all_schemes:
        url = requests.get(data.loc[data['Scheme Name'] == scheme,'scheme_url'].values[0])
        soup = BeautifulSoup(url.text, 'html.parser')

        # Find the script tag with the specific ID
        script_tag = soup.find('script', id='__NEXT_DATA__')

        # Extract the JSON data from the script tag content
        json_data = json.loads(script_tag.contents[0])

        # get holdings
        holdings = json_data['props']['pageProps']['mf']['holdings']

        # get pandas
        hold_df = pd.DataFrame(holdings)
        
        # hold_df = hold_df[['company_name','sector_name','corpus_per']]

        consol_df = pd.concat([hold_df, consol_df], ignore_index=True)

consol_df = consol_df[['company_name','sector_name','corpus_per']]


KeyboardInterrupt: 

In [7]:
print(holdings)

[{'scheme_code': '152202', 'portfolio_date': '2024-02-29T00:00:00.570Z', 'company_name': 'Sun Pharmaceutical Industries Ltd.', 'nature_name': 'EQ', 'sector_name': 'Healthcare', 'instrument_name': 'Equity', 'rating': None, 'market_value': 247.773969, 'corpus_per': 6.07418409, 'market_cap': None, 'rating_market_cap': None, 'stock_search_id': 'sun-pharmaceutical-industries-ltd'}, {'scheme_code': '152202', 'portfolio_date': '2024-02-29T00:00:00.570Z', 'company_name': 'Tata Motors Ltd.', 'nature_name': 'EQ', 'sector_name': 'Automobile', 'instrument_name': 'Equity', 'rating': None, 'market_value': 240.4006, 'corpus_per': 5.89342579, 'market_cap': None, 'rating_market_cap': None, 'stock_search_id': 'tata-motors-ltd'}, {'scheme_code': '152202', 'portfolio_date': '2024-02-29T00:00:00.570Z', 'company_name': 'Mahindra & Mahindra Ltd.', 'nature_name': 'EQ', 'sector_name': 'Automobile', 'instrument_name': 'Equity', 'rating': None, 'market_value': 235.7528, 'corpus_per': 5.77948488, 'market_cap': No

In [74]:
import plotly.express as px
import altair as alt 

# Calculate counts of each category
category_counts = data['Scheme Category Name'].value_counts()

# Create a pie chart
fig = px.pie(values=category_counts.values, names=category_counts.index, title='Pie Chart of Categories')


fig.show()

In [76]:

# Calculate counts of each category
# category_counts = df['category'].value_counts().reset_index()

# Rename columns for clarity
category_counts = data['Scheme Category Name'].value_counts().reset_index()
# Rename columns for clarity
category_counts.columns = ['category', 'count']
alt.Chart(category_counts).mark_arc(innerRadius=50).encode(
    theta="count",
    color="category:N",
)

alt.Chart(...)

In [53]:
import pandas as pd

consol_df = pd.read_csv('./consol_holdings.csv')



In [32]:
import pandas as pd
import numpy as np
import altair as alt

scheme1 = consol_df['Scheme Name'].unique()[0]
scheme2 = consol_df['Scheme Name'].unique()[1]

portfolio1 = consol_df.loc[consol_df['Scheme Name'] == scheme1,['Scheme Name','sector_name','company_name','corpus_per']]
# portfolio1.set_index('company_name',inplace=True)

portfolio2 = consol_df.loc[consol_df['Scheme Name'] == scheme2,['Scheme Name','sector_name','company_name','corpus_per']]

# # sum the corpus_per column for unique sector names
# portfolio2 = portfolio2.groupby('sector_name')['corpus_per'].sum().reset_index()

# portfolio2.set_index('sector_name',inplace=True)

# # sum the corpus_per column for unique sector names
# portfolio1 = portfolio2.groupby('sector_name')['corpus_per'].sum().reset_index()

# portfolio1.set_index('sector_name',inplace=True)


In [65]:
import pandas as pd
import numpy as np
import altair as alt

alt.themes.enable('dark')

consol_df = pd.read_csv('./consol_holdings.csv')
consol_df.columns = ['Scheme Name','Scheme Category', 'Company', 'Sector Name', 'Percent Contribution', 'NAV', 'Units']

scheme1 = consol_df['Scheme Name'].unique()[3]
scheme2 = consol_df['Scheme Name'].unique()[4]

portfolio1 = consol_df.loc[consol_df['Scheme Name'] == scheme1,['Scheme Name','Sector Name','Company','Percent Contribution']]
# portfolio1.set_index('company_name',inplace=True)

portfolio2 = consol_df.loc[consol_df['Scheme Name'] == scheme2,['Scheme Name','Sector Name','Company','Percent Contribution']]

df = pd.concat([portfolio1, portfolio2], axis=0)


# Group by scheme_name and sector, aggregate contribution and unique company names
df_grouped = df.groupby(['Scheme Name', 'Sector Name']).agg({
    'Percent Contribution': 'sum',
    'Company': lambda x: ', '.join(set(x))
}).reset_index()

color_scheme = 'category10'  # You can change the color scheme as needed

# Create the Altair chart
chart = alt.Chart(df_grouped).mark_bar().encode(
    x='Sector Name:N',
    y='sum(Percent Contribution):Q',
    color='Scheme Name:N',
    tooltip=['Sector Name:N', 'Company:N'],
    xOffset=alt.XOffset("Scheme Name:N")
).properties(
    title='Sectorwise Contribution of Schemes',
    width=600,
    height=400
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    grid=False
).configure_legend(
    labelFontSize=12
)


# Display the chart
chart.show()


alt.Chart(...)

In [23]:
from mftool import Mftool

# Create an instance of the Mftool class
mf = Mftool()

# Specify the scheme code
scheme_code = '118989'

# Get the scheme quote
scheme_quote = mf.get_scheme_quote(scheme_code)

# Extract the relevant return values
three_month_return = scheme_quote['three_month_return']
one_year_return = scheme_quote['one_year_return']
five_year_return = scheme_quote['five_year_return']

# Print the results
print(f"3-month return: {three_month_return}%")
print(f"1-year return: {one_year_return}%")
print(f"5-year return: {five_year_return}%")

KeyError: 'three_month_return'

In [36]:
import requests
from bs4 import BeautifulSoup
import json
from mftool import Mftool
mf = Mftool()

scheme_url= 'https://in.investing.com/funds/india-funds'
url = requests.get(scheme_url)
# scrape url
soup = BeautifulSoup(url.text, 'html.parser')

# # Find the script tag with the specific ID
# script_tag = soup.find('script', id='__NEXT_DATA__')
# # Extract the JSON data from the script tag content
# json_data = json.loads(script_tag.contents[0])
# # get holdings
# scheme_code = json_data['props']['pageProps']['mf']['scheme_code']

# data = mf.get_scheme_historical_nav(scheme_code,as_Dataframe=True)

In [1]:
import investpy 

funds = investpy.funds.get_funds(country='india')[['symbol','name']]

In [13]:
funds

,symbol,name
0,0P0000SBP7,Hdfc Standard Life - Balanced Managed Investme...
1,0P0000NQVZ,Icici Prudential Life - Balancer Fund
2,0P0000NNER,Hdfc Standard Life - Balanced Managed Investme...
3,0P0000NNDI,Aditya Birla Sun Life - Individual Creator Fund
4,0P0000NQXG,Icici Prudential Life - Multi Cap Balanced Fund
...,...,...
2978,0P00017T7N,Idfc Bond Fund - Medium Term Plan -regular Pla...
2979,0P00009MZ0,Idfc Bond Fund - Medium Term Plan Regular Plan...
2980,0P00015BYP,L&t Resurgent India Bond Fund Regular Dividend...
2981,0P00005VH3,Idfc Bond Fund - Medium Term Plan Regular Plan...


In [19]:
funds[funds['name'].str.startswith('Hdfc Mid-cap')]


,symbol,name
216,0P00009J3B,Hdfc Mid-cap Opportunities Fund Dividend Payout
217,0P00009J3K,Hdfc Mid-cap Opportunities Fund Growth
218,0P0000XW8E,Hdfc Mid-cap Opportunities Fund -direct Plan D...
219,0P0000XW8F,Hdfc Mid-cap Opportunities Fund -direct Plan -...


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd 


data_scheme = pd.read_csv('https://www.amfiindia.com/spages/NAVAll.TXT',delimiter=';')
data_scheme.dropna(inplace=True)



In [63]:
funds

,symbol,name
0,0P0000SBP7,Hdfc Standard Life - Balanced Managed Investme...
1,0P0000NQVZ,Icici Prudential Life - Balancer Fund
2,0P0000NNER,Hdfc Standard Life - Balanced Managed Investme...
3,0P0000NNDI,Aditya Birla Sun Life - Individual Creator Fund
4,0P0000NQXG,Icici Prudential Life - Multi Cap Balanced Fund
...,...,...
2978,0P00017T7N,Idfc Bond Fund - Medium Term Plan -regular Pla...
2979,0P00009MZ0,Idfc Bond Fund - Medium Term Plan Regular Plan...
2980,0P00015BYP,L&t Resurgent India Bond Fund Regular Dividend...
2981,0P00005VH3,Idfc Bond Fund - Medium Term Plan Regular Plan...


In [73]:
from yahooquery import Ticker
ticker = Ticker('0P0000NQXG.BO')
ticker.fund_top_holdings

symbol              holdingName  holdingPercent
symbol        row                                                        
0P0000NQXG.BO 0      HDFCBANK.NS            HDFC Bank Ltd          0.0416
              1      RELIANCE.NS  Reliance Industries Ltd          0.0283
              2          INFY.NS              Infosys Ltd          0.0267
              3    BAJFINANCE.BO        Bajaj Finance Ltd          0.0204

In [43]:
symbols.iloc[15]

'0P0000XX0C'

In [32]:
from yahoofinancials import YahooFinancials
tickers = ['AAPL', 'GOOG', 'C']
yahoo_financials = YahooFinancials(tickers, concurrent=True, max_workers=8, country="US")
balance_sheet_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'balance')
print(balance_sheet_data_qt)


{'balanceSheetHistoryQuarterly': {'AAPL': [{'2023-03-31': {'currentDebt': 12574000000.0, 'totalEquityGrossMinorityInterest': 62158000000.0, 'inventory': 7482000000.0, 'cashFinancial': 20050000000.0, 'accumulatedDepreciation': -69668000000.0, 'investmentsAndAdvances': 110461000000.0, 'totalDebt': 109615000000.0, 'otherNonCurrentAssets': 65388000000.0, 'tangibleBookValue': 62158000000.0, 'totalLiabilitiesNetMinorityInterest': 270002000000.0, 'cashEquivalents': 4637000000.0, 'otherReceivables': 17963000000.0, 'commercialPaper': 1996000000.0, 'otherCurrentBorrowings': 10578000000.0, 'availableForSaleSecurities': 110461000000.0, 'currentAssets': 112913000000.0, 'currentLiabilities': 120075000000.0, 'grossPPE': 113066000000.0, 'otherEquityAdjustments': -11746000000.0, 'retainedEarnings': 4336000000.0, 'shareIssued': 15723406000.0, 'longTermDebtAndCapitalLeaseObligation': 97041000000.0, 'ordinarySharesNumber': 15723406000.0, 'netPPE': 43398000000.0, 'otherCurrentLiabilities': 56425000000.0, '

In [17]:
import yfinance as yf
import datetime

def get_fiance_ratios(ticker):
    """
    Function to get finance ratios    
    """

   

    # get data for the ticker from yf
    stock_data = yf.Ticker(ticker).history(period='1y')

    # get market cap
    marketcap = stock_data['Close'][-1] * yf.Ticker(ticker).info['sharesOutstanding']

    # get PE ratio; high means over valued low mean under valued
    profit2quity = stock_data['Close'][-1] / yf.Ticker(ticker).info['trailingEps']

    # get price to book ratio; PB=company's assets - liabilities; High P/B ratio overvalued, Low PB ratio mean undervalued.
    book_value = yf.Ticker(ticker).info['bookValue']
    price2book= stock_data['Close'][-1] / book_value

    # get dividend yield; means return on investment from the stock's dividend payments.
    div_yield= yf.Ticker(ticker).info['trailingAnnualDividendYield']

    # Gross margin; percentage of revenue after deducting the cost of producing the goods or services.
    gross_margin = (stock_data['Close'][-1] - stock_data['Open'][0]) / stock_data['Open'][0]

    # return on equity; means return how much a company is making from its investment from shareholders.
    return_equity= yf.Ticker(ticker).info['returnOnEquity']

    # debt to equity; means how much money company is using to finance its assets
    debt2equity = yf.Ticker(ticker).info['debtToEquity']


    return marketcap, profit2quity, price2book, div_yield, gross_margin, return_equity, debt2equity


get_fiance_ratios('RELIANCE.NS')



/tmp/ipykernel_148090/1969698494.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  marketcap = stock_data['Close'][-1] * yf.Ticker(ticker).info['sharesOutstanding']
/tmp/ipykernel_148090/1969698494.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  profit2quity = stock_data['Close'][-1] / yf.Ticker(ticker).info['trailingEps']
/tmp/ipykernel_148090/1969698494.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price2book= stock

(19405204076544.0,
 27.855477855477858,
 2.445465188359654,
 0.003409362,
 0.27535227392508566,
 0.09007,
 37.389)

In [11]:
ticker = 'RELIANCE.NS'
stock_data = yf.Ticker(ticker).history(period='1y')

stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 247 entries, 2023-05-03 00:00:00+05:30 to 2024-05-03 00:00:00+05:30
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          247 non-null    float64
 1   High          247 non-null    float64
 2   Low           247 non-null    float64
 3   Close         247 non-null    float64
 4   Volume        247 non-null    int64  
 5   Dividends     247 non-null    float64
 6   Stock Splits  247 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 15.4 KB


In [4]:
import yfinance as yf

# Define the ticker symbol of the stock to analyze
tickerSymbol = 'AAPL'

# Set the start and end date of the historical data
startDate = '2019-01-01'
endDate = '2022-01-01'

# Retrieve the historical data for the stock
stockData = yf.download(tickerSymbol, startDate, endDate)

# Calculate the financial ratios
marketCap = stockData['Close'].iloc[-1] * stockData.info['sharesOutstanding']
bookValue = stockData.info['bookValue']
peRatio = stockData['Close'].iloc[-1] / stockData.info['trailingEps']
pbRatio = stockData['Close'].iloc[-1] / bookValue
dividendYield =stockData.info['trailingAnnualDividendYield']
grossMargin = (stockData['Close'][-1] - stockData['Open'][0]) / stockData['Open'][0]
returnOnEquity = stockData.info['returnOnEquity']
debtToEquity = stockData.info['debtToEquity']

# Print the financial ratios
print('Market Capitalization:', marketCap)
print('Price-to-Earnings Ratio:', peRatio)
print('Price-to-Book Ratio:', pbRatio)
print('Dividend Yield:', dividendYield)
print('Gross Margin:', grossMargin)

print('Return on Equity:', returnOnEquity)
print('Debt-to-Equity Ratio:', debtToEquity)


[*********************100%***********************]  1 of 1 completed


/home/ankit/miniconda3/envs/DS/lib/python3.11/site-packages/yfinance/utils.py:768: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


TypeError: 'method' object is not subscriptable

In [16]:
from yahoofinancials import YahooFinancials
tickers = ['AAPL']
yahoo_financials = YahooFinancials(tickers, concurrent=True, max_workers=8, country="US")
balance_sheet_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'balance')
print(balance_sheet_data_qt)


{'balanceSheetHistoryQuarterly': {'AAPL': [{'2023-03-31': {'currentDebt': 12574000000.0, 'totalEquityGrossMinorityInterest': 62158000000.0, 'inventory': 7482000000.0, 'cashFinancial': 20050000000.0, 'accumulatedDepreciation': -69668000000.0, 'investmentsAndAdvances': 110461000000.0, 'totalDebt': 109615000000.0, 'otherNonCurrentAssets': 65388000000.0, 'tangibleBookValue': 62158000000.0, 'totalLiabilitiesNetMinorityInterest': 270002000000.0, 'cashEquivalents': 4637000000.0, 'otherReceivables': 17963000000.0, 'commercialPaper': 1996000000.0, 'otherCurrentBorrowings': 10578000000.0, 'availableForSaleSecurities': 110461000000.0, 'currentAssets': 112913000000.0, 'grossPPE': 113066000000.0, 'otherEquityAdjustments': -11746000000.0, 'currentLiabilities': 120075000000.0, 'retainedEarnings': 4336000000.0, 'shareIssued': 15723406000.0, 'longTermDebtAndCapitalLeaseObligation': 97041000000.0, 'ordinarySharesNumber': 15723406000.0, 'netPPE': 43398000000.0, 'otherCurrentLiabilities': 56425000000.0, '

In [23]:
from yahooquery import Ticker
import pandas as pd 

symbol_list_yahoo = ['RELIANCE.NS']

ticker = Ticker(symbol_list_yahoo) 
fin_data_dict = ticker.financial_data
fin_data_df = pd.DataFrame.from_dict(fin_data_dict, orient='index').T

### Scheme Suggest

In [3]:
import mstarpy
import pandas as pd

response = mstarpy.search_funds(term="manufacturing", field=["Name", "fundShareClassId", "GBRReturnM12"], country="in", pageSize=40, currency ="USD")

df = pd.DataFrame(response)
print(df.head())

                                     Name fundShareClassId  GBRReturnM12
0      Aditya BSL Manufacturing Eq Dir Gr       F00000VAVA         48.96
1  Aditya BSL Manufacturing Eq Dir IDCW-P       F00000VAVB         48.24
2  Aditya BSL Manufacturing Eq Dir IDCW-R       F00000Z7IS         48.24
3      Aditya BSL Manufacturing Eq Reg Gr       F00000VAV8         47.65
4  Aditya BSL Manufacturing Eq Reg IDCW-P       F00000VAV9         46.92


In [39]:
import mstarpy

fund = mstarpy.Funds(term="F00000VAVA", country="in")

fund.holdings().head()

,securityName,secId,performanceId,holdingTypeId,weighting,numberOfShare,marketValue,shareChange,country,ticker,...,susEsgRiskGlobes,esgAsOfDate,susEsgRiskCategory,qualRating,quantRating,bestRatingType,securityType,domicileCountryId,currencyName,originalMarketValue
0,Reliance Industries Ltd,0P0000B1W1,None,E,6.24071,183333.0,544811000.0,0.0,India,RELIANCE,...,2.0,2024-05-01T05:00:00.000,High,None,None,None,ST,IND,Indian Rupee,544811000.0
1,Larsen & Toubro Ltd,0P0000C9DZ,None,E,5.38936,125000.0,470488000.0,0.0,India,LT,...,2.0,2024-05-01T05:00:00.000,High,None,None,None,ST,IND,Indian Rupee,470488000.0
2,Bharat Electronics Ltd,0P0000BY3F,None,E,4.96251,2150000.0,433225000.0,-100000.0,India,BEL,...,2.0,2024-05-01T05:00:00.000,High,None,None,None,ST,IND,Indian Rupee,433225000.0
3,Century Textiles & Industries Ltd,0P0000CMM5,None,E,4.21023,225505.0,367551000.0,0.0,India,500040,...,2.0,2024-05-01T05:00:00.000,High,None,None,None,ST,IND,Indian Rupee,367551000.0
4,Cummins India Ltd,0P0000CJ5A,None,E,3.89623,113144.0,340139000.0,0.0,India,500480,...,3.0,2024-05-01T05:00:00.000,Medium,None,None,None,ST,IND,Indian Rupee,340139000.0


In [4]:
from mstarpy import search_funds

search_funds('equity',['Name'], country="in", pageSize=100,)

[{'Name': '360 ONE Focused Equity Dir Gr'},
 {'Name': '360 ONE Focused Equity Dir IDCW-P'},
 {'Name': '360 ONE Focused Equity Dir IDCW-R'},
 {'Name': '360 ONE Focused Equity Reg Gr'},
 {'Name': '360 ONE Focused Equity Reg IDCW-P'},
 {'Name': '360 ONE Focused Equity Reg IDCW-R'},
 {'Name': 'Aditya BSL Equity Advantage Dir Gr'},
 {'Name': 'Aditya BSL Equity Advantage Dir IDCW-P'},
 {'Name': 'Aditya BSL Equity Advantage Dir IDCW-R'},
 {'Name': 'Aditya BSL Equity Advantage Gr'},
 {'Name': 'Aditya BSL Equity Advantage IDCW-P'},
 {'Name': 'Aditya BSL Equity Advantage IDCW-R'},
 {'Name': "Aditya BSL Equity Hybrid '95 Dir Gr"},
 {'Name': "Aditya BSL Equity Hybrid '95 Dir IDCW-P"},
 {'Name': "Aditya BSL Equity Hybrid '95 Dir IDCW-R"},
 {'Name': "Aditya BSL Equity Hybrid '95 Gr"},
 {'Name': "Aditya BSL Equity Hybrid '95 IDCW-P"},
 {'Name': "Aditya BSL Equity Hybrid '95 IDCW-R"},
 {'Name': 'Aditya BSL Equity Savings Dir Gr'},
 {'Name': 'Aditya BSL Equity Savings Dir IDCW-P'},
 {'Name': 'Aditya BS

In [34]:
import mstarpy
fund = mstarpy.Funds(term="F00000VAVA", country="in")

fund.sector()

{'FIXEDINCOME': {'fundPortfolio': {'portfolioDate': '2024-03-31T05:00:00.000',
   'government': 0.0,
   'municipal': 0.0,
   'corporate': 0.0,
   'securitized': 0.0,
   'cashAndEquivalents': 100.0,
   'derivative': 0.0},
  'categoryPortfolio': {'portfolioDate': '2024-03-31T05:00:00.000',
   'government': 3.5065,
   'municipal': 0.0,
   'corporate': 0.10809,
   'securitized': 1.70263,
   'cashAndEquivalents': 94.57491,
   'derivative': 0.10787},
  'indexPortfolio': {'portfolioDate': None,
   'government': None,
   'municipal': None,
   'corporate': None,
   'securitized': None,
   'cashAndEquivalents': None,
   'derivative': None},
  'categoryName': 'Equity - Other',
  'indexName': None,
  'fundName': 'Aditya BSL Manufacturing Eq Dir Gr',
  'assetType': 'FIXEDINCOME'},
 'EQUITY': {'fundPortfolio': {'portfolioDate': '2024-03-31T05:00:00.000',
   'basicMaterials': 21.02485,
   'consumerCyclical': 19.70308,
   'financialServices': 0.0,
   'realEstate': 0.0,
   'communicationServices': 0.0,

In [5]:
import mstarpy
import pandas as pd

response = mstarpy.search_funds(term="", field=["Name",'fundShareClassId'], country="in", pageSize=100000, currency="INR")
df = pd.DataFrame(response)

df.to_parquet('mstar_funds.parquet')

In [13]:
import datetime
import pandas as pd
import mstarpy
# today
today = datetime.date.today()
# yesterday
yesterday = today - datetime.timedelta(days=2)

df_names = pd.read_parquet('mstar_funds.parquet')

fund = mstarpy.Funds(term=df_names['fundShareClassId'].iloc[2000], country="in")


ConnectionError: HTTPSConnectionPool(host='tools.morningstar.co.uk', port=443): Max retries exceeded with url: /api/rest.svc/klr5zyak8x/security/screener?page=1&pageSize=1&sortOrder=LegalName+asc&outputType=json&version=1&universeIds=FOIND%24%24ALL&currencyId=EUR&securityDataPoints=fundShareClassId%7CSecId%7CTenforeId%7CLegalName%7CUniverse&term=F00001J1QV&filters= (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f91955868d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [5]:
fund.name

'Bank of India Multi Asset Allocation Fund Regular Reinvestment Inc Dist cum Cap Wdrl'

In [11]:
holdings = fund.holdings(holdingType="all")
holdings.holdingType.unique()

array(['Equity', 'Bond', 'Other'], dtype=object)

In [12]:
fund.historicalData(fromDate=yesterday, toDate=today).head()

TypeError: Funds.historicalData() got an unexpected keyword argument 'fromDate'

In [9]:
holdings = fund.holdings(holdingType="all")
print(holdings)


ConnectionError: HTTPSConnectionPool(host='api-global.morningstar.com', port=443): Max retries exceeded with url: /sal-service/v1/fund/portfolio/holding/v2/F00001J1QV/data?premiumNum=10000&freeNum=10000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9197205310>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [14]:
import datetime
import pandas as pd
import tqdm 
import mstarpy  # Assuming this is the correct import for the mstarpy library


df_names = pd.read_parquet('mstar_funds.parquet')

active_schemes = []

for fnd in tqdm.tqdm(range(df_names.shape[0])):

    fund = mstarpy.Funds(term=df_names['fundShareClassId'].iloc[fnd], country="in")

    
    i=5
    
    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=i)
    #get historical data
    history = fund.nav(start_date=yesterday,end_date=today,frequency="daily")
    #convert it in pandas DataFrame
    df_history = pd.DataFrame(history)
    if len(df_history) > 0:
        nav = df_history['nav'].iloc[-1]
    else:
        nav = 0
    
    if nav>0:
        active_schemes.append(fnd)


 23%|██▎       | 2514/10832 [42:17<2:19:54,  1.01s/it]


ConnectionError: Error 502
            for the api https://tools.morningstar.co.uk/api/rest.svc/klr5zyak8x/security/screener. Message : Bad Gateway.

In [3]:
import datetime
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import mstarpy  # Assuming this is the correct import for the mstarpy library


df_names = pd.read_parquet('mstar_funds.parquet')

active_schemes = []

def check_active_scheme(fund_id):

    fund = mstarpy.Funds(term=fund_id, country="in")
    i = 5
    # max_days = 3
    nav = 0
    
    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=i)
    # Get historical data
    history = fund.nav(start_date=yesterday, end_date=today,frequency="daily")
    # Convert it into pandas DataFrame
    df_history = pd.DataFrame(history)
    
    if len(df_history) > 0:
        nav = df_history['nav'].iloc[-1]
    
    
    return fund_id if nav > 0 else None

active_schemes = []

for fnd in tqdm(range(df_names.shape[0])):
    future = check_active_scheme(df_names['fundShareClassId'].iloc[fnd])
    if future is not None:
        active_schemes.append(future)

# Now active_schemes contains all active fund IDs

  0%|          | 0/10832 [00:00<?, ?it/s]

  0%|          | 0/10832 [00:00<?, ?it/s]


ConnectionError: Error 403
            for the api https://tools.morningstar.co.uk/api/rest.svc/klr5zyak8x/security/screener. Message : Forbidden.

In [20]:
import mstarpy

# Initialize Morningstar API instance
# morningstar = mstarpy.Morningstar()

# Get the list of mutual funds in India that are active
# This step may vary depending on the exact implementation and data access
mutual_funds = mstarpy.search_funds(country='India', status='active')

# Print the list of active mutual funds
for fund in mutual_funds:
    print(fund)


TypeError: search_funds() got an unexpected keyword argument 'status'

In [19]:
mstarpy.get_funds(country='India', status='active')

AttributeError: module 'mstarpy' has no attribute 'get_funds'